# Pre-Production Context Engine

Copyright 2025, Denis Rothman

*From Prototype to pre-production: Hardening the Context Engine*

This notebook represents a significant architectural evolution from a monolithic prototype to a modular, pre-production system. It no longer contains the core logic itself but instead acts as a high-level "Control Deck" for a powerful multi-agent AI.

The engine's components—utility functions, helper classes, specialized agents, the agent registry, and the core planner/executor—have been refactored into a reusable commons library. These modules are dynamically downloaded from a central GitHub repository, ensuring that this notebook always runs the latest, most stable version of the engine.

The workflow is simple:

**Initialize**: Download the library and set up API clients.

**Configure**: Define all parameters (models, namespaces) in a single config dictionary.

**Execute**: Provide a high-level goal and run the engine.

This modular design promotes maintainability, reusability, and scalability, abstracting away the underlying complexity and allowing the user to focus purely on the task at hand. The output provides both the final generated content and a detailed technical trace for developers.


# I. Inititalization

## GitHub

In [ ]:
# --- Downloading the utilities and the agents ---

# The !curl command is a simple and effective way to download raw files from a public GitHub repo.
# The -L flag ensures that it follows any redirects.

# print("Downloading helper files from public repository...")
# !curl -L https://raw.githubusercontent.com/Denis2054/Context-Engineering/main/commons/utils.py --output utils.py
# !curl -L https://raw.githubusercontent.com/Denis2054/Context-Engineering/main/commons/helpers.py --output helpers.py
# !curl -L https://raw.githubusercontent.com/Denis2054/Context-Engineering/main/commons/agents.py --output agents.py
# !curl -L https://raw.githubusercontent.com/Denis2054/Context-Engineering/main/commons/registry.py.py --output registry.py.py
# !curl -L https://raw.githubusercontent.com/Denis2054/Context-Engineering/main/commons/engine.py --output engine.py

# print("✅ Files downloaded successfully!")

In [ ]:
import requests
import os
from google.colab import userdata

def download_private_github_file(filename, destination_path="."):
    """
    Downloads a file from a private GitHub repository using a token
    stored in Colab Secrets.

    Requires a secret named 'GITHUB_TOKEN' to be set in the Colab UI.
    """
    # --- Configuration: Replace with your repository details ---
    owner = "Denis2054"
    repo = "Context-Engineering"
    branch = "main"
    # ---------------------------------------------------------

    try:
        # Securely fetch the GitHub token from Colab Secrets
        token = userdata.get('GITHUB_TOKEN')
        if not token:
            raise userdata.SecretNotFoundError("Secret 'GITHUB_TOKEN' not found.")
    except userdata.SecretNotFoundError:
        print("🛑 Error: Secret 'GITHUB_TOKEN' not found.")
        print("Please add your GitHub Personal Access Token to the Colab Secrets Manager.")
        return
    except Exception as e:
        print(f"An error occurred while accessing secrets: {e}")
        return

    # Construct the GitHub API URL for the file
    url = f"https://api.github.com/repos/{owner}/{repo}/contents/{filename}?ref={branch}"

    # Prepare headers for authentication and to request the raw file content
    headers = {
        "Authorization": f"Bearer {token}",
        "Accept": "application/vnd.github.v3.raw"
    }

    try:
        # Make the request to the GitHub API
        response = requests.get(url, headers=headers)
        response.raise_for_status()  # This will raise an HTTPError for bad responses (4xx or 5xx)

        # Determine the local filename
        local_filename = os.path.join(destination_path, os.path.basename(filename))

        # Save the file content locally
        with open(local_filename, "wb") as f:
            f.write(response.content)

        print(f"✅ Successfully downloaded '{filename}' to '{local_filename}'")

    except requests.exceptions.HTTPError as e:
        print(f"🛑 Error downloading '{filename}': {e}")
        if e.response.status_code == 404:
            print("   Please check that the owner, repo, file path, and branch are correct.")
        elif e.response.status_code == 401:
            print("   Authentication failed. Please check if your GITHUB_TOKEN is valid and has access to the repo.")
    except Exception as e:
        print(f"An unexpected error occurred: {e}")


# --- Example Usage: Download your utility files ---
# Ensure you have set 'GITHUB_TOKEN' in your Colab Secrets before running this.

download_private_github_file("commons/utils.py")
download_private_github_file("commons/helpers.py")
download_private_github_file("commons/agents.py")
download_private_github_file("commons/registry.py")
download_private_github_file("commons/engine.py")

✅ Successfully downloaded 'commons/utils.py' to './utils.py'
✅ Successfully downloaded 'commons/helpers.py' to './helpers.py'
✅ Successfully downloaded 'commons/agents.py' to './agents.py'
✅ Successfully downloaded 'commons/registry.py' to './registry.py'
✅ Successfully downloaded 'commons/engine.py' to './engine.py'


## Installation and client setup

In [ ]:
#Installation and Client Setup

# Import the setup functions from your new utility file
import utils

# Run the installation
utils.install_dependencies()

# Initialize the OpenAI and Pinecone clients
client, pc = utils.initialize_clients()

🚀 Installing required packages...
✅ All packages installed successfully.

🔑 Initializing API clients...
   - OpenAI client initialized.
   - Pinecone client initialized.
✅ Clients initialized successfully.


## Context Engine library Import

In [ ]:
# 1. Import the hardened helper functions (LLM, Embeddings, Pinecone)
import helpers

# 2. Import the specialist agent functions (Librarian, Researcher, Writer)
import agents

# 3. Import the AGENT_TOOLKIT object that knows about all the agents
from registry import AGENT_TOOLKIT

# 4. Import the main context_engine function that orchestrates the entire process
from engine import context_engine

## Engine Room

In [ ]:
# === ENGINE ROOM: The Main Execution Function ===
# This function contains all the logic to run the engine.
# We define it here so our final cell can be very simple.

import logging
import pprint
from IPython.display import display, Markdown

def execute_and_display(goal, config, client, pc):
    """
    Runs the context engine with a given goal and configuration,
    then displays the final output and the technical trace.
    """
    logging.info(f"******** Starting Engine for Goal: '{goal}' **********\n")

    # 1. Run the Context Engine using the provided configuration
    result, trace = context_engine(
        goal,
        client=client,
        pc=pc,
        **config  # Unpack the config dictionary into keyword arguments
    )

    # 2. Display the Final Result for the main reader
    print("--- FINAL OUTPUT ---")
    if result:
        display(Markdown(result))
    else:
        print(f"The engine failed to produce a result. Status: {trace.status}")

    # 3. Display the Technical Trace for the developer/technical reader
    print("\n\n--- TECHNICAL TRACE (for the tech reader) ---")
    if trace:
        print(f"Trace Status: {trace.status}")
        print(f"Total Duration: {trace.duration:.2f} seconds")
        print("Execution Steps:")
        # Use pprint for a clean, readable dictionary output
        pp = pprint.PrettyPrinter(indent=2)
        pp.pprint(trace.steps)

## Control Deck configuration

In [ ]:
# 1. Define all configuration variables for this run in a dictionary
config = {
    "index_name": 'genai-mas-mcp-ch3',
    "generation_model": "gpt-5",
    "embedding_model": "text-embedding-3-small",
    "namespace_context": 'ContextLibrary',
    "namespace_knowledge": 'KnowledgeStore'
}

#II.CONTROL DECK

=== CONTROL DECK: Define Goal and Run Engine ===
This is the main interactive cell.
1. Change the 'goal' variable to your desired task.
2. Run this cell.


In [ ]:
#Example 1
# Define the high-level goal
goal = "Write a short, suspenseful scene for a children's story about the Apollo 11 moon landing, highlighting the danger."
# Call the execution function from the cell above
execute_and_display(goal, config, client, pc)

--- FINAL OUTPUT ---


July 20, 1969. The cabin hums. The Moon fills the window. Gray. Still. Strange.

I’m in Eagle with Buzz. First try. No second chances.

The small computer chirps. 1202. Then 1201. My chest tightens. We ask. Voices crackle in my headset. Houston comes back fast. “Go.” We keep dropping.

Shadows stretch across the ground. Big. Long. The surface slides under us like a quiet river. Our target waits in the Sea of Tranquility. It feels far and close at once.

Rocks rise where they shouldn’t. A field of boulders near West Crater. Hard teeth. I switch to semi-manual. I take control. I tilt. I drift us past the danger.

The descent engine hisses through the hull. Dust boils up. It climbs and hangs. A pale fog with no wind. It hides the ground. It hides the truth. The Moon keeps its secrets.

Callouts cut the air. Altitude. Speed. Fuel. Each word a drumbeat. “Thirty seconds.” My mouth goes dry. Twenty to thirty seconds of fuel. That’s all. The clock presses close. The unknown presses closer.

I hold a steady hand. I slide us forward. Searching. A smoother patch appears in the haze. I aim for the quiet spot.

“Contact light.” The word lands first. Soft. Then I say it. “Engine stop.” The hiss fades. The dust drifts and settles. The cabin is a whisper.

I breathe. Buzz breathes. Houston waits.

I key the mic. “Houston, Tranquility Base here. The Eagle has landed.”

Silence. Then cheers in my ear. We sit in the stillness. Low fuel. First-time landing. Safe because we worked as one. The danger is outside, in the shadows and dust. The mystery is all around. But for now, we are here.



--- TECHNICAL TRACE (for the tech reader) ---
Trace Status: Success
Total Duration: 148.33 seconds
Execution Steps:
[ { 'agent': 'Librarian',
    'output': { 'blueprint_json': '{"scene_goal": "Increase tension and create '
                                  'suspense.", "style_guide": "Use short, '
                                  'sharp sentences. Focus on sensory details '
                                  '(sounds, shadows). Maintain a slightly '
                                  'eerie but age-appropriate tone.", '
                                  '"participants": [{"role": "Agent", '
                                  '"description": "The protagonist '
                                  'experiencing the events."}, {"role": '
                                  '"Source_of_Threat", "description": "The '
                                  'underlying danger or mystery."}], '
                                  '"instruction": "Rewrite the provided facts '
                              